# Earthquake localization using progressive search grid

In [71]:
!pip install python-dateutil geopy

  Running setup.py bdist_wheel for geographiclib: started
  Running setup.py bdist_wheel for geographiclib: finished with status 'done'
  Stored in directory: C:\Users\ceefour\AppData\Local\pip\Cache\wheels\99\45\d1\14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib


In [1]:
from datetime import datetime
import dateutil.parser

# panyaungan2018
eq_lat = -7.092
eq_lon = 105.963
eq_epicenter = (eq_lat, eq_lon)
eq_depth = 48.2
eq_mw = 6.0
eq_origin_time = dateutil.parser.parse('2018-01-23T06:34:54.000+00:00')

print('EQ origin_time=%s' % (eq_origin_time))


EQ origin_time=2018-01-23 06:34:54+00:00


In [2]:
import pandas as pd

# arrivals = pd.read_csv('panyaungan2018_real.csv')
arrivals = pd.read_csv('panyaungan2018_augmented.csv')
arrivals

,station,net,lat,lon,ele,arrivalTime,epiDistance,azimuth,epiDistanceKm,waveformAvailable
0,CGJI,IA,-6.6100,105.6900,182.00,2018-01-23T06:35:03.254+00:00,0.62,-21.26,NaN,F
1,SKJI,IA,-7.0100,106.5600,100.35,2018-01-23T06:35:03.851+00:00,0.66,73.30,NaN,F
2,DBJI,IA,-6.5500,106.7500,210.84,2018-01-23T06:35:09.672+00:00,1.05,52.18,NaN,F
3,SBJI,IA,-6.1100,106.1300,64.48,2018-01-23T06:35:10.418+00:00,1.10,11.09,NaN,NaN
4,CBJI,IA,-6.7000,106.9300,1006.30,2018-01-23T06:35:10.866+00:00,1.13,63.81,NaN,F
5,CNJI,IA,-7.3100,107.1300,589.00,2018-01-23T06:35:12.060+00:00,1.21,95.46,NaN,F
6,TNGI,IA,-6.1700,106.6500,75.00,2018-01-23T06:35:12.657+00:00,1.25,35.31,NaN,F
7,TNG,IA,-6.1700,106.6500,51.25,2018-01-23T06:35:12.657+00:00,1.25,35.30,NaN,F
8,RBSI,IA,-5.8400,105.7400,224.44,2018-01-23T06:35:14.299+00:00,1.36,-7.39,NaN,F
9,BLSI,IA,-5.3700,105.2500,146.79,2018-01-23T06:35:23.104+00:00,1.95,-20.13,NaN,F


In [3]:
# Parse arrival time
arrivals['arrivalTime_dt'] = arrivals.apply(lambda row: dateutil.parser.parse(row['arrivalTime']), axis=1)
arrivals

,station,net,lat,lon,ele,arrivalTime,epiDistance,azimuth,epiDistanceKm,waveformAvailable,arrivalTime_dt
0,CGJI,IA,-6.6100,105.6900,182.00,2018-01-23T06:35:03.254+00:00,0.62,-21.26,NaN,F,2018-01-23 06:35:03.254000+00:00
1,SKJI,IA,-7.0100,106.5600,100.35,2018-01-23T06:35:03.851+00:00,0.66,73.30,NaN,F,2018-01-23 06:35:03.851000+00:00
2,DBJI,IA,-6.5500,106.7500,210.84,2018-01-23T06:35:09.672+00:00,1.05,52.18,NaN,F,2018-01-23 06:35:09.672000+00:00
3,SBJI,IA,-6.1100,106.1300,64.48,2018-01-23T06:35:10.418+00:00,1.10,11.09,NaN,NaN,2018-01-23 06:35:10.418000+00:00
4,CBJI,IA,-6.7000,106.9300,1006.30,2018-01-23T06:35:10.866+00:00,1.13,63.81,NaN,F,2018-01-23 06:35:10.866000+00:00
5,CNJI,IA,-7.3100,107.1300,589.00,2018-01-23T06:35:12.060+00:00,1.21,95.46,NaN,F,2018-01-23 06:35:12.060000+00:00
6,TNGI,IA,-6.1700,106.6500,75.00,2018-01-23T06:35:12.657+00:00,1.25,35.31,NaN,F,2018-01-23 06:35:12.657000+00:00
7,TNG,IA,-6.1700,106.6500,51.25,2018-01-23T06:35:12.657+00:00,1.25,35.30,NaN,F,2018-01-23 06:35:12.657000+00:00
8,RBSI,IA,-5.8400,105.7400,224.44,2018-01-23T06:35:14.299+00:00,1.36,-7.39,NaN,F,2018-01-23 06:35:14.299000+00:00
9,BLSI,IA,-5.3700,105.2500,146.79,2018-01-23T06:35:23.104+00:00,1.95,-20.13,NaN,F,2018-01-23 06:35:23.104000+00:00


In [4]:
# Sort based on earliest arrival time
arrivals_sorted = arrivals.sort_values('arrivalTime_dt')
arrivals_sorted

,station,net,lat,lon,ele,arrivalTime,epiDistance,azimuth,epiDistanceKm,waveformAvailable,arrivalTime_dt
0,CGJI,IA,-6.6100,105.6900,182.00,2018-01-23T06:35:03.254+00:00,0.62,-21.26,NaN,F,2018-01-23 06:35:03.254000+00:00
1,SKJI,IA,-7.0100,106.5600,100.35,2018-01-23T06:35:03.851+00:00,0.66,73.30,NaN,F,2018-01-23 06:35:03.851000+00:00
2,DBJI,IA,-6.5500,106.7500,210.84,2018-01-23T06:35:09.672+00:00,1.05,52.18,NaN,F,2018-01-23 06:35:09.672000+00:00
3,SBJI,IA,-6.1100,106.1300,64.48,2018-01-23T06:35:10.418+00:00,1.10,11.09,NaN,NaN,2018-01-23 06:35:10.418000+00:00
4,CBJI,IA,-6.7000,106.9300,1006.30,2018-01-23T06:35:10.866+00:00,1.13,63.81,NaN,F,2018-01-23 06:35:10.866000+00:00
5,CNJI,IA,-7.3100,107.1300,589.00,2018-01-23T06:35:12.060+00:00,1.21,95.46,NaN,F,2018-01-23 06:35:12.060000+00:00
6,TNGI,IA,-6.1700,106.6500,75.00,2018-01-23T06:35:12.657+00:00,1.25,35.31,NaN,F,2018-01-23 06:35:12.657000+00:00
7,TNG,IA,-6.1700,106.6500,51.25,2018-01-23T06:35:12.657+00:00,1.25,35.30,NaN,F,2018-01-23 06:35:12.657000+00:00
8,RBSI,IA,-5.8400,105.7400,224.44,2018-01-23T06:35:14.299+00:00,1.36,-7.39,NaN,F,2018-01-23 06:35:14.299000+00:00
10,BBJI,GE,-7.4626,107.6500,615.00,2018-01-23T06:35:19.777+00:00,1.74,98.94,190.6,T,2018-01-23 06:35:19.777000+00:00


In [18]:
arrivals_sorted['latency'] = (arrivals_sorted['arrivalTime_dt'] - eq_origin_time).apply(lambda x: x.total_seconds())

# Cut only 3 nearest stations
arrivals_sorted = arrivals_sorted[:3]
arrivals_sorted

,station,net,lat,lon,ele,arrivalTime,epiDistance,azimuth,epiDistanceKm,waveformAvailable,arrivalTime_dt,latency
0,CGJI,IA,-6.61,105.69,182.00,2018-01-23T06:35:03.254+00:00,0.62,-21.26,NaN,F,2018-01-23 06:35:03.254000+00:00,9.254
1,SKJI,IA,-7.01,106.56,100.35,2018-01-23T06:35:03.851+00:00,0.66,73.30,NaN,F,2018-01-23 06:35:03.851000+00:00,9.851
2,DBJI,IA,-6.55,106.75,210.84,2018-01-23T06:35:09.672+00:00,1.05,52.18,NaN,F,2018-01-23 06:35:09.672000+00:00,15.672


In [19]:
# Earliest station
p1 = arrivals_sorted.iloc[1]
p1

station                                          SKJI
net                                                IA
lat                                             -7.01
lon                                            106.56
ele                                            100.35
arrivalTime             2018-01-23T06:35:03.851+00:00
epiDistance                                      0.66
azimuth                                          73.3
epiDistanceKm                                     NaN
waveformAvailable                                   F
arrivalTime_dt       2018-01-23 06:35:03.851000+00:00
latency                                         9.851
Name: 1, dtype: object

In [20]:
# Arrival time of earliest station
t1 = p1['arrivalTime_dt']
t1

Timestamp('2018-01-23 06:35:03.851000+0000', tz='UTC')

In [21]:
p1_loc = (p1['lat'], p1['lon'])
p1_loc

(-7.01, 106.56)

In [22]:
# Make a grid using 3-deg radius
def round_res(num: float, resolution: float): 
    return round(num / resolution) * resolution

p1_locr = (round_res(p1_loc[0], 0.3), round_res(p1_loc[1], 0.3))
p1_locr

(-6.8999999999999995, 106.5)

In [23]:
import numpy as np

lats = np.arange(p1_locr[0] - 3.0, p1_locr[0] + 3.0 + 0.001, 0.3)
lons = np.arange(p1_locr[1] - 3.0, p1_locr[1] + 3.0 + 0.001, 0.3)

n_points = len(lats) * len(lons)
print('lats=%s lons=%s len=%dx%d n_points=%d' % (lats, lons, len(lats), len(lons), n_points))

grid = np.empty([n_points, 2])
for lat_idx, lat in enumerate(lats):
    for lon_idx, lon in enumerate(lons):
        idx = lat_idx * len(lons) + lon_idx
        grid[idx] = (lat, lon)
grid[:5]

lats=[-9.9 -9.6 -9.3 -9.  -8.7 -8.4 -8.1 -7.8 -7.5 -7.2 -6.9 -6.6 -6.3 -6.
 -5.7 -5.4 -5.1 -4.8 -4.5 -4.2 -3.9] lons=[103.5 103.8 104.1 104.4 104.7 105.  105.3 105.6 105.9 106.2 106.5 106.8
 107.1 107.4 107.7 108.  108.3 108.6 108.9 109.2 109.5] len=21x21 n_points=441


array([[ -9.9, 103.5],
       [ -9.9, 103.8],
       [ -9.9, 104.1],
       [ -9.9, 104.4],
       [ -9.9, 104.7]])

In [24]:
from datetime import timedelta

times = np.empty([31], dtype=datetime)
for idx, sec in enumerate(range(31)):
    times[idx] = t1 - timedelta(seconds=sec)
print(len(times))
times

31


array([Timestamp('2018-01-23 06:35:03.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:35:02.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:35:01.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:35:00.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:34:59.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:34:58.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:34:57.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:34:56.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:34:55.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:34:54.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:34:53.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:34:52.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:34:51.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:34:50.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:34:49.851000+0000', tz='UTC'),
       Timestamp('2018-01-23 06:34:48.851000+0000', tz=

In [182]:
eq_vel_deg = 0.067 # Primary wave velocity = 0.067 degree / second
eq_vel_m = 7437 # Primary wave velocity = 0.067 deg/s * 111000 = 7437 m/s

velocities = np.empty([1])
velocities[0] = eq_vel_m
velocities

array([7437.])

## Weighted RMSE

$$\text{RMSE} = \sqrt{\sum_{i=1}^n w_i (\hat x_i - x_i)^2}$$

In [188]:
# NOTE: This is the "iterative" version for initial experiment and easy changing.
# TODO: Should convert to vector/batch version later, preferably GPU-accelerable.
from geopy import distance

def score_station(station_code, station_loc: (float, float), actual_arrival: datetime,
                  epicenter_e: (float, float), origin_time_e: datetime):
    '''Return distance (m) and the error of arrival time (seconds) for this specific station.'''
    dist_m = distance.distance(epicenter_e, station_loc).m
    travel_dur = dist_m / eq_vel_m
    arrival_time_e = origin_time_e + timedelta(seconds=travel_dur)
    arrival_error = (arrival_time_e - actual_arrival).total_seconds()
#     print('From epicenter %s to station %s %s (%d km) in %s s' % 
#           (epicenter_e, station_code, station_loc, dist_m/1000, travel_dur))
#     print('Est arrival %s actual %s error=%s' % 
#           (arrival_time_e, actual_arrival, arrival_error))
    return (dist_m, arrival_error)

def score(stations, epicenter_e: (float, float), origin_time_e: datetime):
    '''Returns the weighted MSE for all stations given estimates.'''
#     score_station(stations.iloc[0]['station'], (stations.iloc[0]['lat'], stations.iloc[0]['lon']),
#                   stations.iloc[0]['arrivalTime_dt'],
#                   epicenter_e, origin_time_e)
    results = stations.apply(lambda station:
                                 score_station(station['station'], (station['lat'], station['lon']),
                                                 station['arrivalTime_dt'], epicenter_e, origin_time_e)
                            , axis=1)
    result_arr = np.array(list(results))
#     print(result_arr)
    weighted_se = (1000/result_arr[:,0]) * np.square(result_arr[:,1])
#     print(weighted_se)
    weighted_mse = np.mean(weighted_se)
    weighted_rmse = np.sqrt(weighted_mse)
#     print(weighted_rmse)
    return weighted_rmse

score(arrivals_sorted, grid[0], times[0])

2.844131977846634

In [189]:
score(arrivals_sorted, eq_epicenter, eq_origin_time)

0.12833793549102177

In [190]:
# Let's score everything!
combis_loc = []
combis_origin_time_e = []
for loc_e in grid:
    for origin_time_e in times:
        combis_loc.append(tuple(loc_e))
        combis_origin_time_e.append(origin_time_e)
len(combis_origin_time_e)

13671

In [191]:
import pandas as pd

combis = pd.DataFrame({'epicenter_e': combis_loc, 'origin_time_e': combis_origin_time_e})
combis

,epicenter_e,origin_time_e
0,"(-9.899999999999999, 103.5)",2018-01-23 06:35:03.851000+00:00
1,"(-9.899999999999999, 103.5)",2018-01-23 06:35:02.851000+00:00
2,"(-9.899999999999999, 103.5)",2018-01-23 06:35:01.851000+00:00
3,"(-9.899999999999999, 103.5)",2018-01-23 06:35:00.851000+00:00
4,"(-9.899999999999999, 103.5)",2018-01-23 06:34:59.851000+00:00
5,"(-9.899999999999999, 103.5)",2018-01-23 06:34:58.851000+00:00
6,"(-9.899999999999999, 103.5)",2018-01-23 06:34:57.851000+00:00
7,"(-9.899999999999999, 103.5)",2018-01-23 06:34:56.851000+00:00
8,"(-9.899999999999999, 103.5)",2018-01-23 06:34:55.851000+00:00
9,"(-9.899999999999999, 103.5)",2018-01-23 06:34:54.851000+00:00


In [192]:
print('Scoring using %d stations...' % (len(arrivals_sorted)))
combis['wrmse'] = combis.apply(lambda combi: score(arrivals_sorted, combi['epicenter_e'], combi['origin_time_e']), axis=1)
combis

,epicenter_e,origin_time_e,wrmse
0,"(-9.899999999999999, 103.5)",2018-01-23 06:35:03.851000+00:00,2.844132
1,"(-9.899999999999999, 103.5)",2018-01-23 06:35:02.851000+00:00,2.798019
2,"(-9.899999999999999, 103.5)",2018-01-23 06:35:01.851000+00:00,2.751906
3,"(-9.899999999999999, 103.5)",2018-01-23 06:35:00.851000+00:00,2.705795
4,"(-9.899999999999999, 103.5)",2018-01-23 06:34:59.851000+00:00,2.659684
5,"(-9.899999999999999, 103.5)",2018-01-23 06:34:58.851000+00:00,2.613573
6,"(-9.899999999999999, 103.5)",2018-01-23 06:34:57.851000+00:00,2.567464
7,"(-9.899999999999999, 103.5)",2018-01-23 06:34:56.851000+00:00,2.521355
8,"(-9.899999999999999, 103.5)",2018-01-23 06:34:55.851000+00:00,2.475248
9,"(-9.899999999999999, 103.5)",2018-01-23 06:34:54.851000+00:00,2.429141


In [195]:
combis_sorted = combis.sort_values('wrmse')
combis_sorted[:5]

,epicenter_e,origin_time_e,wrmse
6117,"(-7.199999999999992, 105.89999999999998)",2018-01-23 06:34:53.851000+00:00,0.015517
3469,"(-8.399999999999995, 105.29999999999998)",2018-01-23 06:34:35.851000+00:00,0.040539
4793,"(-7.799999999999994, 105.59999999999998)",2018-01-23 06:34:44.851000+00:00,0.045732
4145,"(-8.099999999999994, 105.59999999999998)",2018-01-23 06:34:41.851000+00:00,0.052971
3470,"(-8.399999999999995, 105.29999999999998)",2018-01-23 06:34:34.851000+00:00,0.065581


In [193]:
prediction = combis_sorted[:1].squeeze()
prediction

epicenter_e      (-7.199999999999992, 105.89999999999998)
origin_time_e            2018-01-23 06:34:53.851000+00:00
wrmse                                           0.0155166
Name: 6117, dtype: object

In [201]:
t1_offset = (t1 - eq_origin_time).total_seconds()
epicenter_err = distance.distance(prediction['epicenter_e'], eq_epicenter).km
origin_time_err = abs((prediction['origin_time_e'] - eq_origin_time).total_seconds())
print('Estimated epicenter: %s. Actual epicenter: %s.' % (np.round(np.array(prediction['epicenter_e'])/0.001)*0.001, eq_epicenter))
print('Estimated origin time: %s. Actual origin time: %s.' % (prediction['origin_time_e'], eq_origin_time))
print('Decision latency: T + %s seconds. Epicenter error: ±%d km. Origin time error: ±%s seconds.' % (t1_offset, epicenter_err, origin_time_err))

Estimated epicenter: [ -7.2 105.9]. Actual epicenter: (-7.092, 105.963).
Estimated origin time: 2018-01-23 06:34:53.851000+00:00. Actual origin time: 2018-01-23 06:34:54+00:00.
Decision latency: T + 9.851 seconds. Epicenter error: ±13 km. Origin time error: ±0.149 seconds.
